In [47]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [48]:
# Load API Credentials
with open('/Users/awot1/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [49]:
# set our API call parameters 
LOCATION = 'San Antonio'
TERM = 'ribs'

In [50]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_ribs_san_antonio.json"
JSON_FILE

'Data/results_in_progress_ribs_san_antonio.json'

In [184]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
if file_exists == True:
        
        ## Check if user wants to delete if exists
        
    if delete_if_exists==True:
            
        print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
     ## delete file and confirm it no longer exits.
        os.remove(JSON_FILE)
        
        create_json_file(JSON_FILE,delete_if_exists=False)
    else:
        print(f"[i] {JSON_FILE} already exists.")            
            
            
         ## If it does NOT exist:
else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)

        ## If JSON_FILE included a folder:
        if len(folder)>0:
                os.makedirs(folder,exist_ok=True)
           ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  
                      
            ## If JSON_FILE included a folder:
  
        

[i] Data/results_in_progress_ribs_san_antonio.json not found. Saving empty list to new file.


In [185]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()


dict_keys(['businesses', 'total', 'region'])

In [186]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages



- 0 previous results found.


46

In [187]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    # add a 200ms pause
    time.sleep(.2)



  0%|          | 0/46 [00:00<?, ?it/s]

In [188]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,8MjlOFBianr5LHiOB-X0bQ,the-big-bib-bbq-san-antonio-2,The Big Bib BBQ,https://s3-media1.fl.yelpcdn.com/bphoto/0K5KXC...,False,https://www.yelp.com/biz/the-big-bib-bbq-san-a...,1197,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia...",4.0,"{'latitude': 29.5029421705954, 'longitude': -9...","[delivery, pickup]",$$,"{'address1': '104 Lanark Dr', 'address2': '', ...",+12106548400,(210) 654-8400,9561.838006
1,EjHu7cCILgbhyslPNQt3eA,the-smoke-shack-san-antonio-2,The Smoke Shack,https://s3-media1.fl.yelpcdn.com/bphoto/_NxA1v...,False,https://www.yelp.com/biz/the-smoke-shack-san-a...,1493,"[{'alias': 'southern', 'title': 'Southern'}, {...",4.0,"{'latitude': 29.4610292610233, 'longitude': -9...",[delivery],$$,"{'address1': '3714 Broadway', 'address2': '', ...",+12109571430,(210) 957-1430,5947.265483
2,Hjdt2iBIU1-s5Bah7ygmOQ,2m-smokehouse-bbq-san-antonio,2M Smokehouse BBQ,https://s3-media3.fl.yelpcdn.com/bphoto/mGrYp3...,False,https://www.yelp.com/biz/2m-smokehouse-bbq-san...,532,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia...",4.5,"{'latitude': 29.382481, 'longitude': -98.404915}",[delivery],$$,"{'address1': '2731 S WW White Rd', 'address2':...",+12108859352,(210) 885-9352,16352.264538
3,ggwIaU2YswUXStCB3uOx3w,pinkertons-barbecue-san-antonio-2,Pinkerton's Barbecue,https://s3-media1.fl.yelpcdn.com/bphoto/Puszo4...,False,https://www.yelp.com/biz/pinkertons-barbecue-s...,480,"[{'alias': 'bbq', 'title': 'Barbeque'}]",4.0,"{'latitude': 29.42714866801678, 'longitude': -...",[],$$,"{'address1': '107 W Houston St', 'address2': '...",,,7735.480850
4,Q_DXaBiaG0ITgkuxZj9yvg,zekes-bbq-smokehouse-san-antonio-2,Zeke's BBQ SmokeHouse,https://s3-media1.fl.yelpcdn.com/bphoto/_OnnSn...,False,https://www.yelp.com/biz/zekes-bbq-smokehouse-...,26,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia...",4.5,"{'latitude': 29.45033, 'longitude': -98.57605}","[delivery, pickup]",NaN,"{'address1': '3695 Culebra Rd', 'address2': ''...",+12106003466,(210) 600-3466,7738.081249


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
900,CZN4eC08N7oFECe_OUhsVw,scuzzis-italian-restaurant-san-antonio-4,Scuzzi's Italian Restaurant,https://s3-media4.fl.yelpcdn.com/bphoto/U7SbQC...,False,https://www.yelp.com/biz/scuzzis-italian-resta...,334,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",3.5,"{'latitude': 29.601644, 'longitude': -98.5613288}","[delivery, pickup]",$$,"{'address1': '4035 North Lp W', 'address2': 'S...",+12104938884,(210) 493-8884,12755.679365
901,Zld99JKI_GFHoaxF_khYZQ,genghis-grill-live-oak-2,Genghis Grill,https://s3-media2.fl.yelpcdn.com/bphoto/dG9D2O...,False,https://www.yelp.com/biz/genghis-grill-live-oa...,193,"[{'alias': 'mongolian', 'title': 'Mongolian'},...",2.0,"{'latitude': 29.563067, 'longitude': -98.3276702}","[delivery, pickup]",$$,"{'address1': '8214 Agora Pkwy', 'address2': '8...",+12106585426,(210) 658-5426,19612.344180
902,lde_lt8-TLI6oFOBu8ZZhg,hooters-selma,Hooters,https://s3-media2.fl.yelpcdn.com/bphoto/gQUo8h...,False,https://www.yelp.com/biz/hooters-selma?adjust_...,157,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",2.5,"{'latitude': 29.5784, 'longitude': -98.31562}","[delivery, pickup]",$$,"{'address1': '15412 IH 35 N', 'address2': '', ...",+12109464668,(210) 946-4668,21377.623561
903,IGZLnkBpOqhST4OB8OuxPQ,china-moon-universal-city,China Moon,https://s3-media2.fl.yelpcdn.com/bphoto/s5EclX...,False,https://www.yelp.com/biz/china-moon-universal-...,88,"[{'alias': 'chinese', 'title': 'Chinese'}]",2.5,"{'latitude': 29.549492398733975, 'longitude': ...",[delivery],$,"{'address1': '832 Pat Booker Rd', 'address2': ...",+12106591421,(210) 659-1421,22284.261017
904,_35vfk8EiSZmtFUvC-tQeg,happy-dragon-new-braunfels,Happy Dragon,https://s3-media3.fl.yelpcdn.com/bphoto/eqePmZ...,False,https://www.yelp.com/biz/happy-dragon-new-brau...,221,"[{'alias': 'chinese', 'title': 'Chinese'}, {'a...",3.0,"{'latitude': 29.682311, 'longitude': -98.134965}","[delivery, pickup]",$,"{'address1': '1671 S Interstate 35', 'address2...",+18306099709,(830) 609-9709,42220.143315


In [189]:
## Drop duplicate ids and confirm there are no more duplicates
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()


0